In [22]:
# IMPORTS
from torch import optim
from tqdm.notebook import tqdm, trange
from Build_Histogram import *
from Detect_Feature_And_KeyPoints import *
from Load_Dataset_Folder import *
from Features_Processing import *
from Linear_Processsing_Pipeline import *
from Training_Poly_Processing_Pipeline import *
from Testing_Poly_Processing_Pipeline import *
from calculate_accuracy import *
from train import *
from evaluate import *
from LeNet_Implementation import *
from sklearn.model_selection import train_test_split
import torch.utils.data as data
from VGG_Pretrain import *

## Custom CNN

In [23]:
user= 'perso'
root_path = "C:\\Users\\" + user + "\\Documents\\GitHub\\Shark-Species-Classification"
data_path = os.path.join(root_path, 'Genus Carcharhinus')


dataset = ImageLoader(data_path)

train_dataset, test_dataset = Dataset_Splitter(.5, dataset)
train_train_dataset, validation_dataset = Dataset_Splitter(.9, train_dataset)

In [24]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_train_dataset,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(validation_dataset,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_dataset,
                                batch_size=BATCH_SIZE)

In [ ]:
OUTPUT_DIM = 9
model = AlexNet(OUTPUT_DIM)

In [15]:
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda') #if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

EPOCHS = [1,3,5,10,15,20,30]
EPOCHS = 7

best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'MMnet-model.pt')

    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x968 and 1584x6336)

## VGG Pretrained

In [25]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)

OUTPUT_DIM = 9
model = VGG(vgg11_layers, OUTPUT_DIM)

In [26]:
import torchvision.models as models

pretrained_model = models.vgg11_bn(pretrained=True)

IN_FEATURES = pretrained_model.classifier[-1].in_features
final_fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)

pretrained_model.classifier[-1] = final_fc




In [27]:
model.load_state_dict(pretrained_model.state_dict())

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 128,808,713 trainable parameters


In [36]:
BATCH_SIZE = 128



LR = 1e-6
params = [
          {'params': model.features.parameters(), 'lr': LR / 10},
          {'params': model.classifier.parameters()}
         ]

optimizer = optim.Adam(params, lr=LR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [37]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [38]:

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [39]:

EPOCHS = 7

best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):


    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'VGG-model.pt')


    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 6.00 GiB total capacity; 5.26 GiB already allocated; 0 bytes free; 5.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## SVC & KMeans

In [ ]:

image_files, labels = load_dataset_folder(data_path)
features, processed_labels = Features_Processing(image_files, labels)


features_train, features_test, labels_train, labels_test = train_test_split(features, processed_labels, test_size = .2, random_state = 0)
features_train_train, features_validation, labels_train_train, labels_validation = train_test_split(features_train, labels_train, test_size = .25, random_state = 0)


c, d = Poly_Processing_Pipeline(features_train_train, features_validation, labels_train_train, labels_validation)


training_accuracy = Testing_Poly_SVC(features_train_train, features_validation, labels_train_train, labels_validation, c, d)
testing_accuracy = Testing_Poly_SVC(features_train, features_test, labels_train, labels_test,c, d)

print("c: ", c)
print("d: ", d)
print("Training Accuracy: ", training_accuracy)
print("Testing Accuracy: ", testing_accuracy)
